# sat-search

This notebook is a tutorial on how to use sat-search to search STAC APIs, save the results, and download assets.

Sat-search is built using [sat-stac](https://github.com/sat-utils/sat-stac) which provides the core Python classes used to represent STAC catalogs: `Collection`, `Item`, and `Items`. It is recommended to review the [tutorial on STAC Classes](https://github.com/sat-utils/sat-stac/blob/master/tutorial-2.ipynb) for more information on how to use these objects returned from searching.

Only the `search` module is in sat-search is used as a library, and it contains a single class, `Search`. The `parser` module is used for creating a CLI parser, and `main` contains the main function used in the CLI.

**API endpoint**: Sat-search required an endpoint to be passed in or defined by the STAC_API_URL environment variable. This tutorial uses https://earth-search.aws.element84.com/v0 but any STAC endpoint can be used.

## Initializing a Search object

The first step in performing a search is to create a Search object with all the desired query parameters. Query parameters need to follow the querying as provided in the [STAC specification](https://github.com/radiantearth/stac-spec), although an abbreviated form is also supported (see below).

Another place to look at the STAC query format is in the [sat-api docs](http://sat-utils.github.io/sat-api/), specifically see the section on [full-features querying](http://sat-utils.github.io/sat-api/#search-stac-items-by-full-featured-filtering-) which is what sat-search uses to POST queries to an API. Any field that can be provided in the [searchBody](http://sat-utils.github.io/sat-api/#tocssearchbody) can be provided as a keyword parameter when creating the search. These fields include:

- bbox: bounding box of the form [minlon, minlat, maxlon, maxlat]
- intersects: A GeoJSON geometry
- time: A single date-time, a period string, or a range (seperated by /)
- sort: A dictionary of fields to sort along with ascending/descending
- query: Dictionary of properties to query on, supports eq, lt, gt, lte, gte

Examples of queries are in the sat-api docs, but an example JSON query that would be POSTed might be:

```
{
  "bbox": [
    -110,
    39.5,
    -105,
    40.5
  ],
  "time": "2018-02-12T00:00:00Z/2018-03-18T12:31:12Z",
  "query": {
    "eo:cloud_cover": {
      "lt": 10
    }
  },
  "sort": [
    {
      "field": "eo:cloud_cover",
      "direction": "desc"
    }
  ]
}
```

### Simple queries

In sat-search, each of the fields in the query is simply provided as a keyword argument

In [ ]:
import os
os.environ["STAC_API_URL"] = "https://earth-search.aws.element84.com/v0"
os.getenv('STAC_API_URL')

In [ ]:
from satsearch import Search

In [ ]:
search = Search(bbox=[-76, -7.3, -74, -6.3])

In [ ]:
url = 'https://earth-search.aws.element84.com/v0'

In [ ]:
results = Search(url=url, bbox=[-76, -7.3, -74, -6.3])
print('%s items' % results.found())

### Complex Query

In [ ]:
results = Search(url=url, bbox=[-76, -7.3, -74, -6.3],
               datetime='2017-05-01T00:00:00Z/2019-09-01T00:00:00Z',
               query={'eo:cloud_cover': {'lt': 10}},
               collections=['sentinel-s2-l2a-cogs'],
               sort=['<datetime'])
print('%s items' % results.found())

In [ ]:
items[0].download_assets()

In [ ]:
# save results for later
items = results.items()
items.save('my-s2-l2a-cogs.geojson')

In [ ]:
from satstac import ItemCollection
items = ItemCollection.open('my-s2-l2a-cogs.geojson')

In [ ]:
import geopandas as gpd
gf = gpd.read_file('my-s2-l2a-cogs.geojson')
gf.head(1)

In [ ]:
gf['sentinel:grid_square'].unique()

In [ ]:
gf.tail(1)

In [ ]:
import intake
catalog = intake.open_stac_item_collection(items)

In [ ]:
items.dates()

In [ ]:
import hvplot.xarray

In [ ]:
item = catalog['S2A_18MVT_20190829_0_L2A']

In [ ]:
item.

In [ ]:
dir(item)

In [ ]:
import cartopy

In [ ]:
crs = cartopy.crs.UTM(southern_hemisphere=True, zone=18)

In [ ]:
# Interactive plot with hvplot and datashader (dynamically-updated resolution)
da = item['B06'].to_dask().squeeze(dim='band')
da.hvplot.image(x='x', y='y', crs=crs, tiles='OSM',
                title=f"{item.name} - {item['B06'].description}",
                rasterize=True, frame_width=500)

In [ ]:
da_list = [item['B06'].to_dask().squeeze(dim='band') for item in items]


In [ ]:
item.

In [ ]:
from IPython.display import Image
Image(item['thumbnail'].urlpath)

In [ ]:
da

In [ ]:
bands=['B04','B05','B06']

In [ ]:
import dask
import pandas as pd

@dask.delayed
def stacitem_to_dataset(item):
    print(item.id)
    stacked = catalog[item.id].stack_bands(bands)
    da = stacked(chunks=dict(band=1, x=2048, y=2048)).to_dask()
    da['band'] = bands # use common names
    da = da.expand_dims(time=[pd.to_datetime(item.datetime)])
    ds = da.to_dataset(dim='band')
    return ds

In [ ]:
# concatenate all the MWT images
gfs = gf[gf['sentinel:grid_square'].str.contains('XU')]

In [ ]:
lazy_datasets = []
for i,item in gfs.iterrows():
    ds = stacitem_to_dataset(item)
    lazy_datasets.append(ds)
    
datasets = dask.compute(*lazy_datasets)

In [ ]:
import xarray as xr
ds = xr.concat(datasets, dim='time')

In [ ]:
ds